In [14]:
import os
import pandas as pd
import json
from sklearn.model_selection import train_test_split

def extract_specific_fields_from_json(file_path,folder_path):
    # JSON 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 필요한 필드만 추출
    if "0001. 질의응답" in folder_path:
        filtered_data = {
            "doc_id": data['info'].get('doc_id', None),
            # "casenames": data['info']['casenames'],
            # "casetype": data['info']['casetype'],
            "taskType": data['info']['taskType'],
            "instruction_case": data['taskinfo']['instruction_case'],
            "instruction": data['taskinfo']['instruction'],
            "input": data['taskinfo']['input'],
            "output": data['taskinfo']['output']
        }
    else:
        filtered_data = {
        "doc_id": data['info'].get('caseNum', None),                 # 문서 ID = 사건번호
        "taskType": "QA",                                            # 형사 파일은 고정적으로 QA
        "instruction_case": data['info'].get('caseName', "일반 질의"), # caseName → instruction_case 역할
        "instruction": data['label']['instruction'],
        "input": data['label']['input'],
        "output": data['label']['output']
        }

    
    
    return filtered_data

def read_json_files_from_multiple_folders_to_dataframe(folder_path, target_folder_names):
    # 데이터프레임을 담을 빈 리스트 생성
    data_list = []

    # 지정된 폴더 내의 모든 하위 폴더를 순회
    for root, dirs, files in os.walk(folder_path):
        # 특정 이름을 가진 여러 폴더만 필터링
        # print(os.path.basename(root))
        if os.path.basename(root) in target_folder_names:
            print(os.path.basename(root))
            for file in files:
                if file.endswith(".json"):
                    file_path = os.path.join(root, file)
                    # JSON 파일에서 특정 필드 추출
                    data = extract_specific_fields_from_json(file_path,os.path.basename(root))
                    data['fileName'] = file
                    data_list.append(data)

    # 개별 데이터프레임을 하나의 데이터프레임으로 병합
    combined_df = pd.DataFrame(data_list)
    return combined_df

def folder_test(folder_path, target_folder_names):
    # 데이터프레임을 담을 빈 리스트 생성
    data_frames = []

    # 지정된 폴더 내의 모든 하위 폴더를 순회
    for root, dirs, files in os.walk(folder_path):
        # 특정 이름을 가진 여러 폴더만 필터링
        # print(os.path.basename(root))
        if os.path.basename(root) in target_folder_names:
            print(os.path.basename(root))

In [15]:
# 폴더 경로 및 대상 폴더 이름 목록 지정
law_folder_array=["TL_02. 지식재산권법_001. 판결문_0001. 질의응답","TL_02. 지식재산권법_002. 법령_0001. 질의응답",
                  "TL_02. 지식재산권법_003. 심결례_0001. 질의응답","TL_02. 지식재산권법_004. 심결문_0001. 질의응답",
                 "TL_02. 지식재산권법_005. 유권해석_0001. 질의응답","TL_01. 민사법_001. 판결문_0001. 질의응답",
                  "TL_01. 민사법_002. 법령_0001. 질의응답","TL_01. 민사법_003. 심결례_0001. 질의응답",
                 "TL_01. 민사법_004. 유권해석_0001. 질의응답","TL_결정례_QA","TL_법령_QA",
                 "TL_판결문_QA","TL_해석례_QA"]


In [16]:
folder_path = "F:/sample_database/저작권법_관련_학습데이터"
#for root, dirs, files in os.walk(folder_path):
#    if os.path.basename(root) in target_folder_names:
#        for file in files:
#            print(file)

In [17]:
folder_path = "F:/sample_database/저작권법_관련_학습데이터"
target_folder_names = ["1. 질의응답"]  # 순회할 폴더 이름 목록
combined_df = read_json_files_from_multiple_folders_to_dataframe(folder_path, law_folder_array)

print(len(combined_df))
# 결과 확인
combined_df.head(5)

TL_01. 민사법_001. 판결문_0001. 질의응답
TL_01. 민사법_002. 법령_0001. 질의응답
TL_01. 민사법_003. 심결례_0001. 질의응답
TL_01. 민사법_004. 유권해석_0001. 질의응답
TL_02. 지식재산권법_001. 판결문_0001. 질의응답
TL_02. 지식재산권법_002. 법령_0001. 질의응답
TL_02. 지식재산권법_003. 심결례_0001. 질의응답
TL_02. 지식재산권법_004. 심결문_0001. 질의응답
TL_02. 지식재산권법_005. 유권해석_0001. 질의응답
TL_결정례_QA
TL_법령_QA
TL_판결문_QA
TL_해석례_QA
199218


,doc_id,taskType,instruction_case,instruction,input,output,fileName
0,광주고등법원-2014나2029,01(QA),기타 case,"질의응답을 작성할 때, 예시, 법조문, 법조항 등을 제외하고, 관련된 핵심 정보를 ...","계약상 채무자가 계약을 이행하지 않겠다는 의사를 분명히 표시한 경우, 채권자는 어떤...","계약상 채무자가 계약을 이행하지 않겠다는 의사를 명백히 표시하면, 채권자는 신의성실...",민사법_판결문_질의응답_1.json
1,대구고등법원-2015나1346,01(QA),기타 case,"질의응답을 작성할 때, 예시, 법조문, 법조항 등을 제외하고, 관련된 핵심 정보를 ...",대물변제가 효력을 발생하기 위한 요건은 무엇인가요?,"대물변제가 효력을 발생하려면, 채무자가 본래의 이행에 갈음하여 행하는 다른 급여가 ...",민사법_판결문_질의응답_100.json
2,전주지방법원군산지원-2020가합50961,01(QA),예시문 작성 case,"질의응답을 작성할 때, 예시를 들어 작성함",불가분채무와 연대채무의 차이점은 무엇인가요?,"불가분채무는 채무자가 여러 명일 때 채무의 성질상 나눌 수 없는 채무를 의미하며, ...",민사법_판결문_질의응답_1000.json
3,의정부지방법원-2019가단130210,01(QA),예시문 작성 case,"질의응답을 작성할 때, 예시를 들어 작성함","회사가 근로자에게 작업 환경에서 발생한 사고에 대한 손해배상 책임이 있는 경우, 이...",회사가 작업 환경에서 발생한 사고에 대한 손해배상 책임을 면탈하기 위해서는 근로자에...,민사법_판결문_질의응답_10000.json
4,의정부지방법원-2019가단2070,01(QA),예시문 작성 case,"질의응답을 작성할 때, 예시를 들어 작성함",하도급계약에서 공사 대금 미지급에 따른 지연손해금 계산 절차는 무엇인가요?,하도급계약에서 공사 대금 미지급에 따른 지연손해금 계산은 계약서와 법률에 근거하여 ...,민사법_판결문_질의응답_10001.json


In [18]:
combined_df.to_csv('F:/sample_database/저작권법_관련_학습데이터/law_usloth_data.csv', index=False)

In [19]:
# 데이터 분리
train, temp = train_test_split(combined_df, test_size=0.2, random_state=41)
validation, test = train_test_split(temp, test_size=0.5, random_state=41)

# 데이터셋 저장
train.to_csv('F:/sample_database/저작권법_관련_학습데이터/law_usloth_tran_data.csv', index=False)
validation.to_csv('F:/sample_database/저작권법_관련_학습데이터/law_usloth_validation_data.csv', index=False)
test.to_csv('F:/sample_database/저작권법_관련_학습데이터/law_usloth_test_data.csv', index=False)

In [5]:
print(len(combined_df),'-',len(train),'-',len(validation),'-',len(test))

95200 - 76160 - 9520 - 9520
